In [1]:
import os
os.chdir('../')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="1"

Thu Aug 17 03:07:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  Off  | 00000000:1B:00.0 Off |                    0 |
| N/A   54C    P0    76W / 300W |  57127MiB / 80994MiB |     17%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100 80G...  Off  | 00000000:1C:00.0 Off |                    0 |
| N/A   

### Hyperparams

In [3]:
n_mels = 80
n_outputs = 61
n_frames = 400

### Model

In [4]:
from model.model_ecapa_transformer_reg import Model
from utils.util import *
from tensorboardX import SummaryWriter

step = 0
device = 'cuda:0'

# Model
model = Model(in_dim=n_mels, h_dim=512, out_dim=n_outputs)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
print('done')

done


### Load

In [5]:
save_dir = '/data/scpark/save/lips/train08.16-1/'
!mkdir -p $save_dir
!ls -lt $save_dir

writer = SummaryWriter(save_dir)

if True:
    step, model, _, optimizer = load(save_dir, 7321, model, None, optimizer)

total 2332792
-rw-rw-r-- 1 scpark scpark        40  8월 17 03:07 events.out.tfevents.1692209065.GPUSVR11
-rw-rw-r-- 1 scpark scpark        40  8월 17 03:07 events.out.tfevents.1692209071.GPUSVR11
-rw-rw-r-- 1 scpark scpark      7307  8월 17 03:04 events.out.tfevents.1692195628.GPUSVR11
-rw-rw-r-- 1 scpark scpark 597184874  8월 17 03:01 save_7321
-rw-rw-r-- 1 scpark scpark 597184874  8월 17 00:35 save_2491
-rw-rw-r-- 1 scpark scpark 597176829  8월 16 23:22 save_0
-rw-rw-r-- 1 scpark scpark 597182220  8월 16 23:19 save_58
-rw-rw-r-- 1 scpark scpark       131  8월 16 23:18 events.out.tfevents.1692195270.GPUSVR11
loaded /data/scpark/save/lips/train08.16-1/save_7321


In [6]:
# # warm start
# checkpoint = torch.load('/data/scpark/save/vspeech/train08.14-1/save_80000', map_location=torch.device('cpu'))
# model_state_dict = model.state_dict()

# for key in checkpoint['model_state_dict']:
#     if key in model_state_dict.keys():
#         if checkpoint['model_state_dict'][key].shape == model_state_dict[key].shape:
#             model_state_dict[key] = checkpoint['model_state_dict'][key]
#             print(key)
# model.load_state_dict(model_state_dict, strict=True)
# print('warm start')

### Dataset

In [7]:
import os
from data.arkit_dataset import LipsDataset, CombinedDataset, CombinedCollate

root_dir = '/data/speech/digital_human/preprocessed/'

def get_files(dir):
    data = []
    files = sorted([os.path.join(dir, file) for file in os.listdir(dir)])
    for file in files:
        if file.endswith('.npy') and 'ARKit' in file:
            data.append(file)
        if os.path.isdir(file):
            data.extend(get_files(os.path.join(dir, file)))
    return data

files = get_files(root_dir)
print(len(files))

train_datasets = []
test_datasets = []

def get_sid(file):
    names = ['No Speaker', 'jeewonPark', 'jinwooOh', 'kyuchulLee', 'kyuseokKim', 'nohsikPark', 'soochulPark', 'yehunHwang']
    for sid, name in enumerate(names):
        if name in file:
            return sid
    return 0

for file in files:
    sid = get_sid(file)
    print(sid, file)
    dataset = LipsDataset(file, n_mels, n_frames, sid=sid, mel=False)
    if '_10_' in file:
        test_datasets.append(dataset)
    else:
        train_datasets.append(dataset)
print(len(train_datasets), len(test_datasets))

76
1 /data/speech/digital_human/preprocessed/jeewonPark/MH_ARKit_003_10_iPhone_raw.npy
1 /data/speech/digital_human/preprocessed/jeewonPark/MH_ARKit_003_11_iPhone_raw.npy
1 /data/speech/digital_human/preprocessed/jeewonPark/MH_ARKit_003_12_iPhone_raw.npy
1 /data/speech/digital_human/preprocessed/jeewonPark/MH_ARKit_003_1_iPhone_raw.npy
1 /data/speech/digital_human/preprocessed/jeewonPark/MH_ARKit_003_2_iPhone_raw.npy
1 /data/speech/digital_human/preprocessed/jeewonPark/MH_ARKit_003_3_iPhone_raw.npy
1 /data/speech/digital_human/preprocessed/jeewonPark/MH_ARKit_003_4_iPhone_raw.npy
1 /data/speech/digital_human/preprocessed/jeewonPark/MH_ARKit_003_5_iPhone_raw.npy
1 /data/speech/digital_human/preprocessed/jeewonPark/MH_ARKit_003_8_iPhone_raw.npy
1 /data/speech/digital_human/preprocessed/jeewonPark/MH_ARKit_003_9_iPhone_raw.npy
2 /data/speech/digital_human/preprocessed/jinwooOh/MH_ARKit_005_10_iPhone_raw.npy
2 /data/speech/digital_human/preprocessed/jinwooOh/MH_ARKit_005_1_iPhone_raw.npy
2

In [8]:
train_loader = torch.utils.data.DataLoader(CombinedDataset(train_datasets), 
                                           num_workers=16, shuffle=True, batch_size=32, collate_fn=CombinedCollate())
test_loader = torch.utils.data.DataLoader(CombinedDataset(test_datasets), 
                                          num_workers=10, shuffle=True, batch_size=10, collate_fn=CombinedCollate())
print('done')

done


In [9]:
from functools import partial
from data.audio import mel_spectrogram
get_mel = partial(mel_spectrogram, n_fft=2048, num_mels=80, sampling_rate=24000, hop_size=800, win_size=2048, fmin=0, fmax=None, center=False, return_spec=False)
    

In [ ]:
from IPython import display
import librosa.display
import matplotlib.pyplot as plt

isnan = False
while True:
    if isnan:
        break
    for batch in train_loader:
        inputs = get_mel(torch.Tensor(batch['wav'])).to(device)
        targets = torch.Tensor(batch['blend']).transpose(1, 2).to(device)
        sid = torch.Tensor(batch['sid']).int().to(device)
        
        model.train()
        model.zero_grad()
        outputs = model(inputs, targets, sid)
        
        print(step)
        loss = 0
        for key in outputs.keys():
            if 'loss' in key:
                loss += outputs[key]
                print(key, outputs[key].item())
        if torch.isnan(loss):
            isnan = True
            break
        loss.backward()
        optimizer.step()
        print(step, loss.item())
        
        if step % 100 == 0:
            writer.add_scalar('train_loss', loss.item(), step)
        
        if step % 100 == 0:
            display.clear_output()
            
            # (b, 2)
            speaker = outputs['speaker'].data.cpu().numpy()
            plt.figure(figsize=[8, 8])
            plt.scatter(speaker[:, 0], speaker[:, 1])
            plt.show()
            
            losses = []
            for i, batch in enumerate(test_loader):
                if i >= 10:
                    break
                    
                inputs = get_mel(torch.Tensor(batch['wav'])).to(device)
                targets = torch.Tensor(batch['blend']).transpose(1, 2).to(device)
                sid = torch.Tensor(batch['sid']).int().to(device)

                model.eval()
                with torch.no_grad():
                    outputs = model(inputs, targets, sid)
                    
                loss = 0
                for key in outputs.keys():
                    if 'loss' in key:
                        loss += outputs[key]
                print('test :', i, loss.item())
                losses.append(loss)        
            
            test_loss = torch.stack(losses).mean().item()
            print('test_loss :', test_loss)
            writer.add_scalar('test_loss', test_loss, step)
            
#             plt.figure(figsize=[18, 4])
#             librosa.display.specshow(targets[0].data.cpu().numpy(), cmap='magma')
#             plt.show()

#             plt.figure(figsize=[18, 4])
#             librosa.display.specshow(outputs['y_pred'][0].data.cpu().numpy(), cmap='magma')
#             plt.show()
            
#             for i in [20, 37]:
#                 plt.figure(figsize=[18, 2])
#                 plt.title(str(i))
#                 plt.plot(targets[0].data.cpu().numpy()[i])
#                 plt.plot(outputs['y_pred'][0].data.cpu().numpy()[i])
#                 plt.show()

        if step % 10000 == 0:
            save(save_dir, step, model, None, optimizer)
    
        step += 1

7321
loss 0.027380216866731644
7321 0.027380216866731644
7322
loss 0.028393710032105446
7322 0.028393710032105446
7323
loss 0.030857302248477936
7323 0.030857302248477936
7324
loss 0.02793044224381447
7324 0.02793044224381447
7325
loss 0.027116794139146805
7325 0.027116794139146805
7326
loss 0.02387617714703083
7326 0.02387617714703083
7327
loss 0.025653647258877754
7327 0.025653647258877754
7328
loss 0.02625914104282856
7328 0.02625914104282856
7329
loss 0.031015528365969658
7329 0.031015528365969658
7330
loss 0.027361765503883362
7330 0.027361765503883362
7331
loss 0.029310468584299088
7331 0.029310468584299088
7332
loss 0.026385828852653503
7332 0.026385828852653503
7333
loss 0.027332378551363945
7333 0.027332378551363945
7334
loss 0.026750726625323296
7334 0.026750726625323296
7335
loss 0.02607942931354046
7335 0.02607942931354046
7336
loss 0.02425476163625717
7336 0.02425476163625717
7337
loss 0.023332513868808746
7337 0.023332513868808746
7338
loss 0.029688410460948944
7338 0.029

In [ ]:
save(save_dir, step, model, None, optimizer)

In [ ]:
# (b, 2)
speaker = outputs['speaker'].data.cpu().numpy()
plt.figure(figsize=[8, 8])
plt.scatter(speaker[:, 0], speaker[:, 1])
plt.show()


In [ ]:
print('done')